In [1]:
import pandas as pd
import numpy as np

#Specify filepaths for collecting and saving datasets
filepath_raw = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw CBS data/'
filepath_processed = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Processed CBS data/Population/'

#Population per Municipality per Age
source_population = 'https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=03759ned&_theme=258'
data_population = pd.read_csv(filepath_raw + 'Population_1_jan_raw_cbs_2019.csv', delimiter = ',',low_memory=False)

#file for transforming agecodes
Age_code = pd.read_csv(filepath_raw + 'Age_code.csv', delimiter = ';',low_memory=False,index_col = 0)
#file for transforming municipality codes
Mun_code = pd.read_csv(filepath_raw + 'Gemeenten alfabetisch 2019.csv', delimiter = ',',low_memory=False,index_col = 0)

#Households per Municipality
source_households = 'https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=70072ned&_theme=229'
households = pd.read_csv(filepath_raw + 'huishoudens raw.csv', delimiter = ',', index_col = 0)

#Migration per Municipality
source_migration = 'https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=70072ned&_theme=229'
migration =  pd.read_csv(filepath_raw + 'Gemeente_Migratie_1995-2018.csv', delimiter = ',', index_col = 0)

#Prognosis ABF research 
soure_ABF = 'https://primos.abfresearch.nl/'
prognosisABF = pd.read_csv(filepath_raw + 'Bevolking totaal  - Gemeenten (2018).csv', delimiter = ';', index_col = 0)

In [2]:
df = data_population
#drop unneccesary columns
df = df[['RegioS', 'Leeftijd','Perioden', 'BevolkingOp1Januari_1']]
df.head(10)

,RegioS,Leeftijd,Perioden,BevolkingOp1Januari_1
0,GM1680,10000,2019JJ00,25386.0
1,GM0738,10000,2019JJ00,NaN
2,GM0358,10000,2019JJ00,31728.0
3,GM0197,10000,2019JJ00,27011.0
4,GM0480,10000,2019JJ00,NaN
5,GM0648,10000,2019JJ00,NaN
6,GM0739,10000,2019JJ00,NaN
7,GM0305,10000,2019JJ00,NaN
8,GM0059,10000,2019JJ00,27852.0
9,GM0001,10000,2019JJ00,NaN


In [3]:
df['Perioden'] = df['Perioden'].str.replace('JJ00','')

C:\Users\Dirk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [4]:
leeftijd_dict =Age_code['Title'].to_dict()

In [5]:
df.round({'BevolkingOp1Januari_1':0}).head()

,RegioS,Leeftijd,Perioden,BevolkingOp1Januari_1
0,GM1680,10000,2019,25386.0
1,GM0738,10000,2019,NaN
2,GM0358,10000,2019,31728.0
3,GM0197,10000,2019,27011.0
4,GM0480,10000,2019,NaN


In [6]:
df[['BevolkingOp1Januari_1']] = pd.to_numeric(df.BevolkingOp1Januari_1, errors = 'coerce')#.fillna(0)

C:\Users\Dirk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Dirk\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
df.Leeftijd= df.Leeftijd.replace(leeftijd_dict)

C:\Users\Dirk\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [8]:
df.head()

,RegioS,Leeftijd,Perioden,BevolkingOp1Januari_1
0,GM1680,Totaal,2019,25386.0
1,GM0738,Totaal,2019,NaN
2,GM0358,Totaal,2019,31728.0
3,GM0197,Totaal,2019,27011.0
4,GM0480,Totaal,2019,NaN


In [9]:
Mun_code_dict = Mun_code[['GemeentecodeGM','Gemeentenaam']].set_index(['GemeentecodeGM']).to_dict()['Gemeentenaam']

In [10]:
#df.loc[:,'RegioS'] = df.RegioS.replace(Mun_code_dict)
df.RegioS =  df.RegioS.replace(Mun_code_dict)

In [11]:
df.Leeftijd = pd.to_numeric(df['Leeftijd'].str.replace(" jaar",'').str.replace("Totaal",'0.1').str.replace("95 of ouder","0.95").str.replace("of ouder",""), errors = 'ignore')

In [12]:
df = df[(df.Leeftijd < 95)] 


In [13]:
df.tail()

,RegioS,Leeftijd,Perioden,BevolkingOp1Januari_1
80505,GM0192,0.95,2019,NaN
80506,GM0138,0.95,2019,NaN
80507,Zwijndrecht,0.95,2019,71.0
80508,Zwolle,0.95,2019,165.0
80509,GM0997,0.95,2019,NaN


In [14]:
df = df[df.RegioS.str.contains("GM") == False]

In [15]:
group_dict = {0.1:"Totaal", 0.95: '80plus'}
x = 4
for i in range (98):
    if i>x:
        x+=5
    y = x - 4
    group_dict[i] = 'Age' + str(y) + 'to' + str(x)
df['Leeftijdsgroep'] = df.Leeftijd.map(group_dict)
group_dict2 = {'Age80to84':'80plus','Age85to89':'80plus','Age90to94':'80plus'}
df['Leeftijdsgroep'][df['Leeftijd']>79] = df.Leeftijdsgroep.map(group_dict2)
total_population = df

C:\Users\Dirk\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [16]:
#df = df.sort_index().rename(columns = {'RegioS':'Municipality', 'GemiddeldeBevolking_2':'Population','Leeftijdsgroep':'Age Group Population'}).drop('Leeftijd', axis = 1)
df[df.RegioS == 'Aa en Hunze'][df.Leeftijdsgroep == 'Totaal']

C:\Users\Dirk\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,RegioS,Leeftijd,Perioden,BevolkingOp1Januari_1,Leeftijdsgroep
0,Aa en Hunze,0.1,2019,25386.0,Totaal


In [17]:
df = df.sort_index().groupby(by = ('RegioS',"Leeftijdsgroep")).sum().reset_index().rename(columns = {'RegioS':'Municipality', 'BevolkingOp1Januari_1':'Population','Leeftijdsgroep':'Age Group Population'}).drop('Leeftijd', axis = 1)

C:\Users\Dirk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [18]:
df_pivot = df.pivot(index='Municipality', columns='Age Group Population', values='Population')

In [19]:
df_pivot = df_pivot[['Age0to4','Age5to9','Age10to14','Age15to19','Age20to24','Age25to29','Age30to34','Age35to39', 'Age40to44','Age45to49','Age50to54','Age55to59','Age60to64','Age65to69','Age70to74','Age75to79','80plus','Totaal']]

In [20]:
df_pivot.fillna(0).head()

Age Group Population,Age0to4,Age5to9,Age10to14,Age15to19,Age20to24,Age25to29,Age30to34,Age35to39,Age40to44,Age45to49,Age50to54,Age55to59,Age60to64,Age65to69,Age70to74,Age75to79,80plus,Totaal
Municipality,,,,,,,,,,,,,,,,,,
's-Gravenhage,31560.0,31247.0,29725.0,30613.0,35486.0,42320.0,41602.0,40413.0,37690.0,40652.0,37031.0,33430.0,28044.0,24176.0,21409.0,13505.0,18930.0,537833.0
's-Hertogenbosch,7750.0,8162.0,8030.0,8624.0,9499.0,11129.0,10736.0,9933.0,9355.0,11448.0,11428.0,10994.0,9625.0,8740.0,7567.0,4920.0,6265.0,154205.0
Aa en Hunze,922.0,1174.0,1366.0,1579.0,1031.0,912.0,1002.0,1124.0,1309.0,1919.0,2077.0,2208.0,2150.0,2061.0,1850.0,1196.0,1506.0,25386.0
Aalsmeer,1506.0,1970.0,2246.0,2046.0,1633.0,1473.0,1588.0,1888.0,2124.0,2756.0,2686.0,2252.0,1730.0,1518.0,1695.0,1158.0,1459.0,31728.0
Aalten,1197.0,1415.0,1682.0,1685.0,1345.0,1293.0,1276.0,1397.0,1449.0,2048.0,2131.0,2201.0,1928.0,1707.0,1656.0,1138.0,1463.0,27011.0


In [21]:
#gevuld met nullen voor lege cellen: betere oplossing bedenken
df_pivot.to_csv(filepath_processed + 'Population_Municipalities_2019.csv')

In [22]:
df_pivot.sort_values (by = 'Totaal').head()

Age Group Population,Age0to4,Age5to9,Age10to14,Age15to19,Age20to24,Age25to29,Age30to34,Age35to39,Age40to44,Age45to49,Age50to54,Age55to59,Age60to64,Age65to69,Age70to74,Age75to79,80plus,Totaal
Municipality,,,,,,,,,,,,,,,,,,
Schiermonnikoog,26.0,32.0,45.0,40.0,43.0,47.0,35.0,55.0,54.0,73.0,74.0,74.0,79.0,66.0,74.0,52.0,67.0,936.0
Vlieland,34.0,36.0,51.0,68.0,88.0,95.0,59.0,55.0,62.0,97.0,106.0,109.0,66.0,63.0,66.0,42.0,41.0,1138.0
Rozendaal,71.0,102.0,127.0,110.0,62.0,22.0,38.0,81.0,97.0,136.0,127.0,141.0,118.0,105.0,121.0,77.0,119.0,1654.0
Ameland,168.0,198.0,219.0,164.0,217.0,235.0,210.0,206.0,197.0,265.0,254.0,251.0,261.0,244.0,259.0,161.0,164.0,3673.0
Terschelling,171.0,194.0,217.0,295.0,478.0,287.0,235.0,242.0,252.0,369.0,362.0,362.0,299.0,327.0,348.0,211.0,241.0,4890.0


In [23]:
#households.Perioden = households['Perioden'].str.replace('JJ00','')

In [24]:
households.RegioS = households.RegioS.replace(Mun_code_dict)

In [25]:
households = households[households.RegioS.str.contains("GM") == False]

In [26]:
households = households.sort_values(by = 'RegioS').fillna(2.17).reset_index()#

In [27]:
households = households.drop(['Perioden','ID'], axis = 1).rename(columns={'RegioS':'Municipality','GemiddeldeHuishoudensgrootte_89':'Average size Household'})

In [28]:
households.head()

,Municipality,Average size Household
0,'s-Gravenhage,2.01
1,'s-Hertogenbosch,2.09
2,Aa en Hunze,2.23
3,Aalsmeer,2.40
4,Aalten,2.33


In [29]:
households = households.set_index('Municipality')[['Average size Household']]

In [30]:
households.transpose().to_csv(filepath_processed + 'Households_Municipalities_2019.csv')

In [31]:
migration.head()

,RegioS,Perioden,BinnenlandsMigratiesaldoRelatief_72,MigratiesaldoRelatief_77
ID,,,,
1425,GM1680,1995JJ00,NaN,NaN
1426,GM1680,1996JJ00,NaN,NaN
1427,GM1680,1997JJ00,NaN,NaN
1428,GM1680,1998JJ00,-13.0,7.5
1429,GM1680,1999JJ00,-5.7,6.2


In [32]:
migration.RegioS = migration.RegioS.replace(Mun_code_dict)
migration = migration[migration.RegioS.str.contains("GM") == False]
migration.Perioden = pd.to_numeric(migration.Perioden.str.replace('JJ00',''))
migration = migration.rename(columns = {'RegioS': 'Municipality', 'Perioden': 'Year', 'BinnenlandsMigratiesaldoRelatief_72':'Relative Domestic Migration','MigratiesaldoRelatief_77': 'Relative Foreign Migration'})


In [33]:
migration = migration[migration.Year > 1994]
migration.head()

,Municipality,Year,Relative Domestic Migration,Relative Foreign Migration
ID,,,,
1425,Aa en Hunze,1995,NaN,NaN
1426,Aa en Hunze,1996,NaN,NaN
1427,Aa en Hunze,1997,NaN,NaN
1428,Aa en Hunze,1998,-13.0,7.5
1429,Aa en Hunze,1999,-5.7,6.2


In [34]:
#make pivot table and alter from per 1000 to per 1
migration_domestic = migration.pivot(index = 'Municipality' , columns = 'Year' , values = 'Relative Domestic Migration').fillna(0)/1000
migration_foreign = migration.pivot(index = 'Municipality', columns = 'Year', values = 'Relative Foreign Migration').fillna(0)/1000

In [35]:
migration_total = migration_foreign + migration_domestic

In [36]:
def extrapolate (df, order):
    #extrapolation
    from scipy.optimize import curve_fit

    # Function to curve fit to the data
    
    if order == 4:
        def func(x, a, b, c, d):
            return a * (x ** 3) + b * (x ** 2) + c * x + d
    if order == 3:    
        def func(x, a, b, c):
            return a * (x ** 2) + b * x + c
    if order == 2:    
        def func(x, a, b):
            return a * x + b

    # Initial parameter guess, just to kick off the optimization
    guess = (0.5, 0.5, 0.5, 0.5)
    #guess = (0.5, 0.5, 0.5)
    #guess = (0.5, 0.5)
    #guess = (0.5, 0.5, 0.5, 0.5, 0.5)

    # Create copy of data to remove NaNs for curve fitting
    fit_df = df.dropna()
    
    # Place to store function parameters for each column
    col_params = {}

    # Curve fit each column
    for col in fit_df.columns:
        # Get x & y
        x = fit_df.index.astype(float).values
        y = fit_df[col].values
        # Curve fit column and get curve parameters
        params = curve_fit(func, x, y, maxfev=1000000) #p0=guess
        # Store optimized parameters

        col_params[col] = params[0]

    # Extrapolate each column
    for col in df.columns:
        # Get the index values for NaNs in the column
        x = df[pd.isnull(df[col])].index.astype(float).values
        # Extrapolate those points with the fitted function
        df[col][x]  = func(x, *col_params[col])
        output = df
    return output

In [37]:
for i in range (2019,2061):
    migration_domestic[i] = np.NaN
    migration_foreign[i] = np.NaN
migration_domestic_extr = extrapolate(migration_domestic.transpose(),2)
migration_foreign_extr = extrapolate(migration_foreign.transpose(),2)

In [38]:
#migration_domestic_extr

In [39]:
migration_foreign.transpose().head()

Municipality,'s-Gravenhage,'s-Hertogenbosch,Aa en Hunze,Aalsmeer,Aalten,Achtkarspelen,Alblasserdam,Albrandswaard,Alkmaar,Almelo,...,Zeist,Zevenaar,Zoetermeer,Zoeterwoude,Zuidplas,Zundert,Zutphen,Zwartewaterland,Zwijndrecht,Zwolle
Year,,,,,,,,,,,,,,,,,,,,,
1995,0.0068,0.0040,0.0000,0.0007,0.0059,-0.0001,0.0012,-0.0019,0.0024,0.0017,...,0.0053,0.0000,0.0014,0.0006,0.0,-0.0016,0.0020,0.0,0.0004,0.0003
1996,0.0052,0.0042,0.0000,0.0018,0.0104,0.0008,0.0005,-0.0018,0.0013,0.0033,...,0.0020,-0.0001,0.0000,-0.0005,0.0,-0.0025,0.0008,0.0,-0.0002,0.0012
1997,0.0061,0.0018,0.0000,0.0004,0.0114,-0.0002,-0.0005,-0.0007,0.0020,0.0029,...,-0.0004,-0.0003,-0.0007,0.0005,0.0,-0.0001,0.0020,0.0,0.0003,0.0003
1998,0.0003,0.0030,0.0075,-0.0004,0.0050,0.0006,0.0011,0.0006,0.0044,0.0034,...,0.0055,-0.0008,0.0020,0.0020,0.0,-0.0004,0.0003,0.0,0.0013,0.0020
1999,0.0050,0.0020,0.0062,0.0010,0.0061,-0.0001,0.0020,-0.0002,0.0029,0.0029,...,0.0032,0.0011,0.0014,0.0009,0.0,0.0010,0.0018,0.0,0.0019,0.0020


In [40]:
migration_foreign_est = migration_foreign_extr.reset_index()[migration_foreign_extr.reset_index().Year > 2018].set_index('Year')
migration_domestic_est = migration_domestic_extr.reset_index()[migration_domestic_extr.reset_index().Year > 2018].set_index('Year')


In [41]:
migration_foreign_est = migration_foreign_est
migration_domestic_est = migration_domestic_est

migration_foreign_est.to_csv(filepath_processed + 'Migration Foreign Estimate.csv')
migration_domestic_est.to_csv(filepath_processed + 'Migration Domestic Estimate.csv')

In [42]:
prognosis = prognosisABF
prognosis = prognosis.reset_index().rename(columns = {'Gemeenten (2018)': 'Municipality','Bevolking totaal|2018':2018,'Bevolking totaal|2019':2019,'Bevolking totaal|2020' : 2020 , 'Bevolking totaal|2025' :2025,'Bevolking totaal|2030':2030,'Bevolking totaal|2035':2035,'Bevolking totaal|2040':2040,'Bevolking totaal|2045':2045,'Bevolking totaal|2050':2050 })
prognosis = prognosis.set_index('Municipality')

In [43]:
for x in range (6):
    a = x*5 + 2020
    for y in range (4):
        b = a + y +1
        prognosis[b] = np.NaN
prognosis = prognosis.transpose()#.sort_index(axis = 0)#.interpolate(axis = 1)
prognosis.index = prognosis.index.astype(int)
prognosis = prognosis.sort_index().interpolate().transpose()

In [44]:
prognosis_relative = prognosis[[]]

In [45]:
for i in range (2019,2051):
    prognosis_relative[i]= prognosis[i]/prognosis[i-1]
  

C:\Users\Dirk\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [46]:
prognosis_relative.head()

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
Municipality,,,,,,,,,,,,,,,,,,,,,
Aa en Hunze,1.000000,0.998425,0.996292,0.996278,0.996264,0.996250,0.996236,0.996463,0.996450,0.996438,...,0.996975,0.996966,0.996956,0.996947,0.996938,0.995734,0.995716,0.995697,0.995678,0.995660
Aalburg,1.012167,1.004508,1.005984,1.005948,1.005913,1.005878,1.005844,1.004648,1.004626,1.004605,...,0.999163,0.999162,0.999161,0.999160,0.999160,0.998739,0.998737,0.998735,0.998734,0.998732
Aalsmeer,1.007302,1.003782,1.002575,1.002568,1.002561,1.002555,1.002548,1.001860,1.001856,1.001853,...,1.003245,1.003235,1.003224,1.003214,1.003204,1.003785,1.003770,1.003756,1.003742,1.003728
Aalten,1.001855,0.994817,0.997990,0.997986,0.997982,0.997978,0.997974,0.998271,0.998268,0.998265,...,0.994676,0.994647,0.994619,0.994590,0.994560,0.993421,0.993377,0.993333,0.993288,0.993243
Achtkarspelen,0.997853,0.997131,0.996043,0.996027,0.996012,0.995996,0.995980,0.996477,0.996465,0.996452,...,0.995421,0.995400,0.995379,0.995357,0.995336,0.994917,0.994891,0.994864,0.994838,0.994811


In [47]:
migration_total = migration_total.drop(2019,axis = 1)    

In [48]:
combined = pd.concat([prognosis_relative -1, migration_total],join = 'outer', axis = 1).sort_index(axis = 1)
combined.loc["'s-Gravenhage"] = combined.loc['Den Haag'].fillna(0) + combined.loc["'s-Gravenhage"].fillna(0)
combined = combined.drop(combined[combined[2018].isnull()].index)

for i in range (2051,2061):
    combined[i] = np.NaN
combined = combined.transpose()
combined.index = combined.index.astype(int)
combined = combined[combined.index>2010]
combined[combined.index > 2049] = combined[combined.index > 2049].ffill()

C:\Users\Dirk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [49]:
combined.tail(12)

,'s-Gravenhage,'s-Hertogenbosch,Aa en Hunze,Aalsmeer,Aalten,Achtkarspelen,Alblasserdam,Albrandswaard,Alkmaar,Almelo,...,Zeist,Zevenaar,Zoetermeer,Zoeterwoude,Zuidplas,Zundert,Zutphen,Zwartewaterland,Zwijndrecht,Zwolle
2049,0.006126,-0.000354,-0.004322,0.003742,-0.006712,-0.005162,0.001955,0.002804,-0.001649,-0.004439,...,0.002462,-0.004274,0.005429,0.003886,0.005151,0.0,-0.003543,-0.001014,-0.000128,-0.000635
2050,0.006088,-0.000354,-0.004340,0.003728,-0.006757,-0.005189,0.001951,0.002796,-0.001652,-0.004459,...,0.002455,-0.004293,0.005399,0.003871,0.005125,0.0,-0.003556,-0.001015,-0.000128,-0.000636
2051,0.006088,-0.000354,-0.004340,0.003728,-0.006757,-0.005189,0.001951,0.002796,-0.001652,-0.004459,...,0.002455,-0.004293,0.005399,0.003871,0.005125,0.0,-0.003556,-0.001015,-0.000128,-0.000636
2052,0.006088,-0.000354,-0.004340,0.003728,-0.006757,-0.005189,0.001951,0.002796,-0.001652,-0.004459,...,0.002455,-0.004293,0.005399,0.003871,0.005125,0.0,-0.003556,-0.001015,-0.000128,-0.000636
2053,0.006088,-0.000354,-0.004340,0.003728,-0.006757,-0.005189,0.001951,0.002796,-0.001652,-0.004459,...,0.002455,-0.004293,0.005399,0.003871,0.005125,0.0,-0.003556,-0.001015,-0.000128,-0.000636
2054,0.006088,-0.000354,-0.004340,0.003728,-0.006757,-0.005189,0.001951,0.002796,-0.001652,-0.004459,...,0.002455,-0.004293,0.005399,0.003871,0.005125,0.0,-0.003556,-0.001015,-0.000128,-0.000636
2055,0.006088,-0.000354,-0.004340,0.003728,-0.006757,-0.005189,0.001951,0.002796,-0.001652,-0.004459,...,0.002455,-0.004293,0.005399,0.003871,0.005125,0.0,-0.003556,-0.001015,-0.000128,-0.000636
2056,0.006088,-0.000354,-0.004340,0.003728,-0.006757,-0.005189,0.001951,0.002796,-0.001652,-0.004459,...,0.002455,-0.004293,0.005399,0.003871,0.005125,0.0,-0.003556,-0.001015,-0.000128,-0.000636
2057,0.006088,-0.000354,-0.004340,0.003728,-0.006757,-0.005189,0.001951,0.002796,-0.001652,-0.004459,...,0.002455,-0.004293,0.005399,0.003871,0.005125,0.0,-0.003556,-0.001015,-0.000128,-0.000636
2058,0.006088,-0.000354,-0.004340,0.003728,-0.006757,-0.005189,0.001951,0.002796,-0.001652,-0.004459,...,0.002455,-0.004293,0.005399,0.003871,0.005125,0.0,-0.003556,-0.001015,-0.000128,-0.000636


In [50]:
combined_ext = extrapolate(combined,2)

In [51]:
combined_ext = combined_ext[combined_ext.index > 2018]

In [52]:
combined_ext.to_csv(filepath_processed + 'Total prognosis migration.csv')